### Intro
Walking through this example from a [Medium article](https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916)

In [117]:
# In the terminal, install the ipywidets package using:
# conda install -c conda-forge ipywidgets

# NOTE: only need to do this once

In [2]:
import ipywidgets as widgets

### Adding a slider

In [8]:
# Define a min and max value, a step size, a description and an initial value
widgets.IntSlider(
    min = 0,
    max = 10,
    step = 1,
    description = 'Slider:',
    value = 5
)

IntSlider(value=5, description='Slider:', max=10)

If we assign the slider to a variable, then it won't display automatically. The display() function will render the widget in that case

In [5]:
from IPython.display import display

In [9]:
slider = widgets.IntSlider(
    min = 1,
    max = 15,
    step = 1,
    description = 'Slider:',
    value = 3        
)
display(slider)

IntSlider(value=3, description='Slider:', max=15, min=1)

The value of a widget is stored in its "value" property

In [10]:
slider.value

3

Synchronise two widgets by using the jslink() function

In [12]:
# First widget, e.g. a slider
slider = widgets.IntSlider()
# Second widget, e.g a text box
text = widgets.IntText()
# Display them both
display(slider, text)
# Link the values together, so they are synchronised i.e. show the same values
# NOTE: if we exceed the max value of the slider in the text box, the max value will be shown instead
widgets.jslink((slider, 'value'), (text, 'value'))

IntSlider(value=0)

IntText(value=0)

In [13]:
# Full list of widgets:
print(dir(widgets))

['Accordion', 'Audio', 'BoundedFloatText', 'BoundedIntText', 'Box', 'Button', 'ButtonStyle', 'CallbackDispatcher', 'Checkbox', 'Color', 'ColorPicker', 'Controller', 'CoreWidget', 'DOMWidget', 'DatePicker', 'Datetime', 'Dropdown', 'FloatLogSlider', 'FloatProgress', 'FloatRangeSlider', 'FloatSlider', 'FloatText', 'GridBox', 'HBox', 'HTML', 'HTMLMath', 'Image', 'IntProgress', 'IntRangeSlider', 'IntSlider', 'IntText', 'Label', 'Layout', 'NumberFormat', 'Output', 'Password', 'Play', 'RadioButtons', 'Select', 'SelectMultiple', 'SelectionRangeSlider', 'SelectionSlider', 'SliderStyle', 'Style', 'Tab', 'Text', 'Textarea', 'ToggleButton', 'ToggleButtons', 'ToggleButtonsStyle', 'VBox', 'Valid', 'ValueWidget', 'Video', 'Widget', '__builtins__', '__cached__', '__doc__', '__file__', '__jupyter_widgets_base_version__', '__jupyter_widgets_controls_version__', '__loader__', '__name__', '__package__', '__path__', '__protocol_version__', '__spec__', '__version__', '_handle_ipython', '_version', 'dlink', 

### Triggering events
Depending on the specific features of a widget, there are different events that can be executed when the event is triggered.

For example, we can create a button that will do something when clicked, like print some text

In [15]:
btn = widgets.Button(description = 'Anna Jones')
display(btn)

def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))

btn.on_click(btn_eventhandler)

Button(description='Anna Jones', style=ButtonStyle())

Hello from the Anna Jones button!
Hello from the Anna Jones button!
Hello from the Anna Jones button!


### Filtering a dataframe

In [16]:
# Download the data
import pandas as pd
import numpy as np
url = "https://data.london.gov.uk/download/number-international-visitors-london/b1e0f953-4c8a-4b45-95f5-e0d143d5641e/international-visitors-london-raw.csv"
df_london = pd.read_csv(url)

Take a sample so the output is truncated

In [22]:
df_london = df_london.sample(250)

Here we'll make a function called unique_values_plus_ALL, which will return all the the unique values to filter on, sort them, and then add the ALL option in case we don't want to filter

In [23]:
ALL = 'ALL'
def unique_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [24]:
# Make the dropdown of years to filter the dataframe on later
dropdown_year = widgets.Dropdown(options = unique_values_plus_ALL(df_london.year))

"The dropdown widget exposes the observe method, which takes a function that will be invoked when the value of the dropdown changes. 

We create the observer handler to filter the dataframe by the selected values — note that the input argument of the handler, change, contains information about the changes that took place which allows us to access the new value (change.new)"

In [25]:
def dropdown_year_eventhandler(change):
    if (change.new == ALL):
        display(df_london)
    else:
        display(df_london[df_london.year == change.new])

In [40]:
dropdown_year.observe(dropdown_year_eventhandler, names='value')

In [28]:
display(dropdown_year)

Dropdown(options=('ALL', 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2…

We really don't want to print a new dataframe every time we change the value, we just want to refresh.
The solution is to capture the cell output in a special kind of widget, namely Output, and then display it in another cell.

In [55]:
dropdown_year = widgets.Dropdown(options = unique_values_plus_ALL(df_london.year))

# create a new instance of Output
output_year = widgets.Output()

# call the clear_output method within the event handler to clear the previous selection on each iteration
# and capture the output of the dataframe in a with block.
def dropdown_year_eventhandler(change):
    output_year.clear_output()
    with output_year:
        if (change.new == ALL):
            display(df_london)
        else:
            display(df_london[df_london.year == change.new])
dropdown_year.observe(dropdown_year_eventhandler, names='value')
display(dropdown_year)

Dropdown(options=('ALL', 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2…

In [56]:
display(output_year)

Output()

### Filter on multiple conditions
Let’s assume we would also like to filter by purpose too. Right now, the dataframe only responds to the most recently changed filter

In [57]:
# First, we need a common output
output = widgets.Output()

In [58]:
# Then we make the two dropdowns
dropdown_year = widgets.Dropdown(options = unique_values_plus_ALL(df_london.year))
dropdown_purpose = widgets.Dropdown(options = unique_values_plus_ALL(df_london.purpose))

In [59]:
# Then we create a new function, common_filtering, that will be called by both the event handlers. 
# This function will apply a filter on the dataframe for both year AND purpose:

In [60]:
def common_filtering(year, purpose):
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & 
                                  (df_london.purpose == purpose)]
    
    with output:
        display(common_filter)

In [61]:
# We amend the event handlers to call the common_filtering function 
# and pass the change.new value as well as the current value of the other dropdown:

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value)
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new)

In [62]:
# We bind the handlers to the dropdowns, and that’s it!
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')

In [63]:
display(dropdown_year)
display(dropdown_purpose)

Dropdown(options=('ALL', 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2…

Dropdown(options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value='ALL')

In [64]:
display(output)

Output()

### Dashboarding
Now that we've filtered the dataframe, we move on by colouring the numeric values based on a user selected value

In [65]:
# A useful numeric widget is the BoundedFloatText; 
# We give it a min, max and initial value, and the incremental step

In [66]:
bounded_num = widgets.BoundedFloatText(
    min = 0, 
    max = 100000, 
    value = 5, 
    step = 1)

In [67]:
# In order to colour the dataframe cells, we will define this function:

def colour_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'

In [86]:
# Amend the common_filtering method with the num input parameter
# and apply the styling by calling the colour_ge_value function for the three numeric columns

def common_filtering(year, purpose, num):
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & 
                                  (df_london.purpose == purpose)]
    
    with output:
        display(common_filter.style.applymap(lambda x: colour_ge_value(x, num), subset=['visits','spend', 'nights']))

In [87]:
# The existing event handlers need to be adjusted to pass the bounded_num.value
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value, bounded_num.value)
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, bounded_num.value)

In [88]:
# And finally we will plug-in the event handler of the new widget:
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)

In [89]:
dropdown_year = widgets.Dropdown(options = unique_values_plus_ALL(df_london.year), description = "Year")
dropdown_purpose = widgets.Dropdown(options = unique_values_plus_ALL(df_london.purpose), description = "Purpose")
bounded_num = widgets.BoundedFloatText(min = 0, max = 100000, value = 5, step = 1, description = "Number")

In [90]:
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

In [91]:
display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)

Dropdown(description='Year', options=('ALL', 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,…

Dropdown(description='Purpose', options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value…

BoundedFloatText(value=5.0, description='Number', max=100000.0, step=1.0)

In [92]:
display(output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<pandas.io.formats.style.Styler at 0x1…

### Plotting

In [93]:
import seaborn as sns
import matplotlib.pyplot as plt

In [105]:
# We capture the plot in a new variable
plot_output = widgets.Output()

In [97]:
# We will now amend the common_filtering function to plot the new diagram:
# - first we clear the output
# - then we call the kdeplot method of seaborn by passing the number of visits
# - Lastly, the only thing we need to do is to display the outputs in a new cell:

In [106]:
# Amend the common_filtering method with the num input parameter
# and apply the styling by calling the colour_ge_value function for the three numeric columns

def common_filtering(year, purpose, num):
    output.clear_output()
    plot_output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & 
                                  (df_london.purpose == purpose)]
    
    with output:
        display(common_filter.style.applymap(lambda x: colour_ge_value(x, num), subset=['visits','spend', 'nights']))
        
    with plot_output:
        sns.kdeplot(common_filter['visits'], shade=True)
        plt.show()

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value, bounded_num.value)
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, bounded_num.value)
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)

dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)

Dropdown(description='Year', index=4, options=('ALL', 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 20…

Dropdown(description='Purpose', index=3, options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR…

BoundedFloatText(value=50.0, description='Number', max=100000.0, step=1.0)

In [107]:
display(output)
display(plot_output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<pandas.io.formats.style.Styler at 0x1…

Output()

### Dashboard layout

In [109]:
# We will first arrange the input widgets horizontally. 
# The HBox will add widgets to it one at a time from left-to-right:
input_widgets = widgets.HBox([dropdown_year, dropdown_purpose, bounded_num])
display(input_widgets)

In [111]:
# Next we will create a container for the output. 
# Tab is great for this. The 1st tab will host the dataframe and the 2nd one the graph.
tab = widgets.Tab([output, plot_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')
display(tab)

In [112]:
# Now we combine the two
dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)

In [116]:
# And add some spacing
item_layout = widgets.Layout(margin='0 0 50px 0')
input_widgets = widgets.HBox([dropdown_year, dropdown_purpose, bounded_num], layout=item_layout)
tab = widgets.Tab([output, plot_output], layout=item_layout)
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')
dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)